In [ ]:
!pip install -q groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.5/103.5 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 714.3 kB/s eta 0:00:00


In [ ]:
GROQ_KEY="add your key"

In [ ]:
import json
import datetime
import os

from groq import Groq
date = datetime.datetime.now()
date
client = Groq(
    api_key=GROQ_KEY,
)
MODEL = 'llama3-70b-8192'

In [ ]:
# %%writefile conversation_functions.py
GROQ_KEY="add your key"
import json
import datetime
import os

from groq import Groq
date = datetime.datetime.now()
date
client = Groq(
    api_key=GROQ_KEY,
)
MODEL = 'llama3-70b-8192'

sys_prompt = """
You are a chatbot for a pizza company. Your tasks are:

1. **Greet the Customer**: Start the conversation with a friendly greeting.
2. **Provide Pizza Options**: Present available pizzas and their prices. Ask the customer to choose their pizza and any extras.
3. **Collect Delivery Address**: Ask for the delivery address and confirm it.
4. **Confirm the Order**: Summarize the pizza order details and confirm with the customer.
5. **Inform About Charges**: Mention any delivery fees.
6. **Final Confirmation**: Review the complete order and ask for final confirmation before placing it.

**Guidelines**:
- Focus on one task at a time.
- Be clear and direct.
- Try to respond consise and to the point without additional details
- Ensure all details are correct before finalizing the order.
"""

import json

pizza_menu = {
    "Margherita": {"description": "Classic pizza with tomato sauce and mozzarella cheese", "price": 8.99},
    "Pepperoni": {"description": "Pizza topped with pepperoni slices", "price": 9.99},
    "Veggie Supreme": {"description": "Loaded with vegetables and cheese", "price": 10.99},
    "BBQ Chicken": {"description": "Grilled chicken with BBQ sauce", "price": 11.99}
}

delivery_charges = {
    "Downtown": 5.00,
    "Suburbs": 7.50,
    "Countryside": 10.00
}

def greet_customer():
    return "Welcome the customer"

def show_pizza_menu():
    return json.dumps(pizza_menu, indent=4)

def show_pizza_details(pizza_name):
    pizza = pizza_menu.get(pizza_name)
    if pizza:
        return json.dumps(pizza, indent=4)
    else:
        return "Pizza not found."

def show_pizza_price(pizza_name):
    pizza = pizza_menu.get(pizza_name)
    if pizza:
        return f"The price of {pizza_name} is ${pizza['price']:.2f}."
    else:
        return "Pizza not found."

# def request_delivery_location():
#     return "Please provide your delivery location so we can calculate the delivery charges."

def confirm_delivery_location(location):
    if location in delivery_charges:
        return f"Delivery location confirmed: {location}. Delivery charges will be calculated based on this location."
    else:
        return "Location not recognized. Please provide a valid location."

def calculate_delivery_charges(location):
    if location in delivery_charges:
        return f"Delivery charges for {location} are ${delivery_charges[location]:.2f}."
    else:
        return "Location not recognized. Delivery charges cannot be calculated."

def process_order(pizza_name, quantity, location):
    if pizza_name not in pizza_menu:
        return "Pizza not found. Please choose a valid pizza from the menu."

    if location not in delivery_charges:
        return "Location not recognized. Please provide a valid delivery location."

    total_price = pizza_menu[pizza_name]['price'] * quantity
    delivery_charge = delivery_charges[location]
    total_amount = total_price + delivery_charge

    return f"Order placed for {quantity} {pizza_name}(s). Total amount: ${total_amount:.2f} (including delivery charge of ${delivery_charge:.2f})."

# Define tools for the pizza ordering system
TOOLS = [
    {
        "type": "function",
        "function": {
            "name": "greet_customer",
            "description": "Greet the customer",
            "parameters": {
                "type": "object",
                "properties": {}
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "show_pizza_menu",
            "description": "Show the complete menu of available pizzas with descriptions and prices",
            "parameters": {
                "type": "object",
                "properties": {}
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "show_pizza_details",
            "description": "Show detailed information about a specific pizza",
            "parameters": {
                "type": "object",
                "properties": {
                    "pizza_name": {
                        "type": "string",
                        "description": "Name of the pizza to get details about"
                    }
                },
                "required": ["pizza_name"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "show_pizza_price",
            "description": "Show the price of a specific pizza",
            "parameters": {
                "type": "object",
                "properties": {
                    "pizza_name": {
                        "type": "string",
                        "description": "Name of the pizza to get the price of"
                    }
                },
                "required": ["pizza_name"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "request_delivery_location",
            "description": "Ask the customer for their delivery location",
            "parameters": {
                "type": "object",
                "properties": {}
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "confirm_delivery_location",
            "description": "Confirm the delivery location provided by the customer",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "Delivery location provided by the customer"
                    }
                },
                "required": ["location"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "calculate_delivery_charges",
            "description": "Calculate and return the delivery charges based on the location",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "Location to calculate the delivery charges for"
                    }
                },
                "required": ["location"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "process_order",
            "description": "Process the pizza order and calculate the total amount including delivery charges",
            "parameters": {
                "type": "object",
                "properties": {
                    "pizza_name": {
                        "type": "string",
                        "description": "Name of the pizza to order"
                    },
                    "quantity": {
                        "type": "integer",
                        "description": "Quantity of the pizza to order"
                    },
                    "location": {
                        "type": "string",
                        "description": "Delivery location"
                    }
                },
                "required": ["pizza_name", "quantity", "location"]
            }
        }
    }
]

messages = [
    {
        "role": "system",
        "content": sys_prompt
    },
]
def run_conversation(user_prompt):
    messages.append({
        "role": "user",
        "content": user_prompt
    })
    response = client.chat.completions.create(
        model=MODEL,
        messages=messages,
        tools=TOOLS,
        tool_choice="auto",
        max_tokens=4096,
        temperature=0,

    )
    response_message = response.choices[0].message
    tool_calls = response_message.tool_calls
    print(tool_calls)
    if tool_calls:
        available_functions = {
            "greet_customer":greet_customer,
            "show_pizza_menu": show_pizza_menu,
            "show_pizza_details": show_pizza_details,
            "show_pizza_price": show_pizza_price,
            # "request_delivery_location":request_delivery_location,
            "confirm_delivery_location":confirm_delivery_location,
            "calculate_delivery_charges":calculate_delivery_charges,
            "process_order":process_order
        }

        for tool_call in tool_calls:
            function_name = tool_call.function.name
            function_to_call = available_functions.get(function_name)

            if function_to_call:
                print(function_name)
                function_args = json.loads(tool_call.function.arguments)
                function_response = function_to_call(**function_args)
                messages.append(
                    {
                        "tool_call_id": tool_call.id,
                        "role": "tool",
                        "name": function_name,
                        "content": function_response,
                    }
                )

        second_response = client.chat.completions.create(
            model=MODEL,
            messages=messages
        )
        # if messages:
        #     messages.pop()
        response_message = second_response.choices[0].message

    # else:
    messages.append({
        "role":"assistant",
        "content": response_message.content
    })
    return response_message.content
# user_prompt = "hello"
# print(run_conversation(user_prompt))

In [ ]:
%%writefile conversation_functions.py
GROQ_KEY="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"
import json
import datetime
import os

from groq import Groq
date = datetime.datetime.now()
date
client = Groq(
    api_key=GROQ_KEY,
)
MODEL = 'llama3-70b-8192'

sys_prompt = """
You are a chatbot for a pizza company. Your tasks are:

1. **Greet the Customer**: Start the conversation with a friendly greeting.
2. **Provide Pizza Options**: Present available pizzas and their prices. Ask the customer to choose their pizza and any extras.
3. **Collect Delivery Address**: Ask for the delivery address and confirm it.
4. **Confirm the Order**: Summarize the pizza order details and confirm with the customer.
5. **Inform About Charges**: Mention any delivery fees.
6. **Final Confirmation**: Review the complete order and ask for final confirmation before placing it.

**Guidelines**:
- Focus on one task at a time.
- Be clear and direct.
- Try to respond consise and to the point without additional details
- Ensure all details are correct before finalizing the order.
"""

import json

pizza_menu = {
    "Margherita": {"description": "Classic pizza with tomato sauce and mozzarella cheese", "price": 8.99},
    "Pepperoni": {"description": "Pizza topped with pepperoni slices", "price": 9.99},
    "Veggie Supreme": {"description": "Loaded with vegetables and cheese", "price": 10.99},
    "BBQ Chicken": {"description": "Grilled chicken with BBQ sauce", "price": 11.99}
}

delivery_charges = {
    "Downtown": 5.00,
    "Suburbs": 7.50,
    "Countryside": 10.00
}

def greet_customer():
    return "Welcome the customer"

def show_pizza_menu():
    return json.dumps(pizza_menu, indent=4)

def show_pizza_details(pizza_name):
    pizza = pizza_menu.get(pizza_name)
    if pizza:
        return json.dumps(pizza, indent=4)
    else:
        return "Pizza not found."

def show_pizza_price(pizza_name):
    pizza = pizza_menu.get(pizza_name)
    if pizza:
        return f"The price of {pizza_name} is ${pizza['price']:.2f}."
    else:
        return "Pizza not found."

def request_delivery_location():
    return "Please provide your delivery location so we can calculate the delivery charges."

def confirm_delivery_location(location):
    if location in delivery_charges:
        return f"Delivery location confirmed: {location}. Delivery charges will be calculated based on this location."
    else:
        return "Location not recognized. Please provide a valid location."

def calculate_delivery_charges(location):
    if location in delivery_charges:
        return f"Delivery charges for {location} are ${delivery_charges[location]:.2f}."
    else:
        return "Location not recognized. Delivery charges cannot be calculated."

def process_order(pizza_name, quantity, location):
    if pizza_name not in pizza_menu:
        return "Pizza not found. Please choose a valid pizza from the menu."

    if location not in delivery_charges:
        return "Location not recognized. Please provide a valid delivery location."

    total_price = pizza_menu[pizza_name]['price'] * quantity
    delivery_charge = delivery_charges[location]
    total_amount = total_price + delivery_charge

    return f"Order placed for {quantity} {pizza_name}(s). Total amount: ${total_amount:.2f} (including delivery charge of ${delivery_charge:.2f})."

# Define tools for the pizza ordering system
TOOLS = [
    {
        "type": "function",
        "function": {
            "name": "greet_customer",
            "description": "Greet the customer",
            "parameters": {
                "type": "object",
                "properties": {}
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "show_pizza_menu",
            "description": "Show the complete menu of available pizzas with descriptions and prices",
            "parameters": {
                "type": "object",
                "properties": {}
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "show_pizza_details",
            "description": "Show detailed information about a specific pizza",
            "parameters": {
                "type": "object",
                "properties": {
                    "pizza_name": {
                        "type": "string",
                        "description": "Name of the pizza to get details about"
                    }
                },
                "required": ["pizza_name"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "show_pizza_price",
            "description": "Show the price of a specific pizza",
            "parameters": {
                "type": "object",
                "properties": {
                    "pizza_name": {
                        "type": "string",
                        "description": "Name of the pizza to get the price of"
                    }
                },
                "required": ["pizza_name"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "request_delivery_location",
            "description": "Ask the customer for their delivery location",
            "parameters": {
                "type": "object",
                "properties": {}
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "confirm_delivery_location",
            "description": "Confirm the delivery location provided by the customer",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "Delivery location provided by the customer"
                    }
                },
                "required": ["location"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "calculate_delivery_charges",
            "description": "Calculate and return the delivery charges based on the location",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "Location to calculate the delivery charges for"
                    }
                },
                "required": ["location"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "process_order",
            "description": "Process the pizza order and calculate the total amount including delivery charges",
            "parameters": {
                "type": "object",
                "properties": {
                    "pizza_name": {
                        "type": "string",
                        "description": "Name of the pizza to order"
                    },
                    "quantity": {
                        "type": "integer",
                        "description": "Quantity of the pizza to order"
                    },
                    "location": {
                        "type": "string",
                        "description": "Delivery location"
                    }
                },
                "required": ["pizza_name", "quantity", "location"]
            }
        }
    }
]

messages = [
    {
        "role": "system",
        "content": sys_prompt
    },
]
def run_conversation(user_prompt):
    messages.append({
        "role": "user",
        "content": user_prompt
    })
    response = client.chat.completions.create(
        model=MODEL,
        messages=messages,
        tools=TOOLS,
        tool_choice="auto",
        max_tokens=4096,
        temperature=0,

    )
    response_message = response.choices[0].message
    tool_calls = response_message.tool_calls
    print(tool_calls)
    if tool_calls:
        available_functions = {
            "greet_customer":greet_customer,
            "show_pizza_menu": show_pizza_menu,
            "show_pizza_details": show_pizza_details,
            "show_pizza_price": show_pizza_price,
            # "request_delivery_location":request_delivery_location,
            "confirm_delivery_location":confirm_delivery_location,
            "calculate_delivery_charges":calculate_delivery_charges,
            "process_order":process_order
        }

        for tool_call in tool_calls:
            function_name = tool_call.function.name
            function_to_call = available_functions.get(function_name)

            if function_to_call:
                print(function_name)
                function_args = json.loads(tool_call.function.arguments)
                function_response = function_to_call(**function_args)
                messages.append(
                    {
                        "tool_call_id": tool_call.id,
                        "role": "tool",
                        "name": function_name,
                        "content": function_response,
                    }
                )

        second_response = client.chat.completions.create(
            model=MODEL,
            messages=messages
        )
        # if messages:
        #     messages.pop()
        response_message = second_response.choices[0].message

    # else:
    messages.append({
        "role":"assistant",
        "content": response_message.content
    })
    return response_message.content
# user_prompt = "hello"
# print(run_conversation(user_prompt))

Overwriting conversation_functions.py


In [ ]:
!pip install streamlit pyngrok


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 50.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 61.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.0/83.0 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 4.0 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.0.0
    Uninstalling tenacity-9.0.0:
      Successfully uninstalled tenacity-9.0.0


In [ ]:
%%writefile app.py
import streamlit as st
from conversation_functions import run_conversation

def main():
    st.title("Pizza Ordering")

    if 'chat_history' not in st.session_state:
        st.session_state.chat_history = []

    for user_msg, bot_msg in st.session_state.chat_history:

        if user_msg:
            st.markdown(f"""
            <div style="display: flex; justify-content: flex-start; margin-bottom: 10px;">
                <div style="background-color: #d1e7dd; border-radius: 10px; padding: 10px; max-width: 60%; color: #0f5132; text-align: left;">
                    {user_msg}
                </div>
            </div>
            """, unsafe_allow_html=True)

        # Bot messages
        if bot_msg:
            st.markdown(f"""
            <div style="display: flex; justify-content: flex-end; margin-bottom: 10px;">
                <div style="background-color: #f8d7da; border-radius: 10px; padding: 10px; max-width: 90%; color: #721c24; text-align: right;">
                    {bot_msg}
                </div>
            </div>
            """, unsafe_allow_html=True)

    user_input = st.text_input("Your message:")


    if st.button("Send"):
        if user_input:
            bot_response = run_conversation(user_input)
            st.session_state.chat_history.append((user_input, bot_response))

    # Button to clear chat history
    if st.button("Clear Chat History"):
        st.session_state.chat_history = []

if __name__ == "__main__":
    main()


In [ ]:
import os
from pyngrok import ngrok

port = 8501  

public_url = ngrok.connect(port)
print(f"Streamlit app is live at: {public_url}")

os.system(f'streamlit run app.py --server.port {port}')


Streamlit app is live at: NgrokTunnel: "https://4c48-35-247-65-191.ngrok-free.app" -> "http://localhost:8501"


2